In [64]:
from urllib.request import urlopen
from urllib.error import HTTPError
from urllib.error import URLError

import bs4
from bs4 import BeautifulSoup as bs

In [72]:
import difflib
from pprint import pprint
import re

In [3]:
target_sites = ['http://www.coolenjoy.net/bbs/40', 'https://ddd.com'] # DC예수게이 갤러리 안된다. 

In [4]:
def error_checker(site):
    try:
        html = urlopen(site)
    except HTTPError as e:
        print(site, "The server returned an HTTP error, ", e)
        return 1
    except URLError as e:
        print(site, "The server could not be found. ", e)
        return 1
    else: # Success
        return 0

In [5]:
filtered_target_sites = list(map(lambda x: x if error_checker(x)==0 else None, target_sites))
filtered_target_sites = list(filter(None, filtered_target_sites))
filtered_target_sites

https://ddd.com The server could not be found.  <urlopen error [Errno 11001] getaddrinfo failed>


['http://www.coolenjoy.net/bbs/40']

In [8]:
html = urlopen(site)
html_bs = bs(html, 'html.parser')

In [70]:
html_bs.descendents == None # 왜 descendents는 비어있는걸까? 

True

In [55]:
soup = bs('<b class="boldest">Extremely bold</b>')
tag = soup.b
type(tag)

bs4.element.Tag

## 사이트의 한 게시글을 특정하기

In [78]:
cooln = html_bs.find_all(text=re.compile('오큘'))
cooln

['\r\n                    오큘 리프트s 블랙아웃 현상 때문에 고통받고 있네요                    ',
 '\r\n                    오큘러스 고를 받고 살짝 후회가 듭니다                    ',
 '\r\n                    오큘러스 리프트 S 구매해습니다.                    ']

In [80]:
ocul = cooln[0]
type(cooln[0])

bs4.element.NavigableString

In [93]:
ocul.parent.parent.parent.parent.parent.parent.parent.parent.parent.parent.parent # <!DOCTYPE HTML> 까지 가면 bs object itself가 되고, 그보다 parent는 None이다. 

In [95]:
for parent in ocul.parents:
    if parent is None:
        print(parent)
    else:
        print(parent.name)

a
td
tr
tbody
table
div
form
div
div
[document]


In [100]:
ocul.parent.parent.parent.parent.find_all(text=re.compile('오큘'))

['\r\n                    오큘 리프트s 블랙아웃 현상 때문에 고통받고 있네요                    ',
 '\r\n                    오큘러스 고를 받고 살짝 후회가 듭니다                    ',
 '\r\n                    오큘러스 리프트 S 구매해습니다.                    ']

## 한 게시글을 알았을 때, sibling을 구하기

In [101]:
def find_common_ancestor(child):
    for parent in child.parents:
        if len(parent.find_all(text=re.compile('오큘'))) == 1:
            continue
        if len(parent.find_all(text=re.compile('오큘'))) > 1:
            return parent
        else:
            print('Something went wrong.')
            return

In [106]:
common_ancestor = find_common_ancestor(ocul)
common_ancestor.name

'tbody'

In [114]:
ocul.parent.parent.parent.parent.name
common_ancestor_son = ocul.parent.parent.parent # commen ancestor 직후의 ancestor
common_ancestor_son.name

'tr'

In [115]:
common_ancestor.find_all('{}'.format(common_ancestor_son.name))

[<tr class="">
 <td class="td_num">
 <i aria-hidden="true" class="fa fa-question"></i> 질문            </td>
 <td class="td_subject">
 <a href="http://www.coolenjoy.net/bbs/40/7819">
                     오디세이G9 존버중인데 제 컴사양 너무 구릴까요?                                    </a>
 </td>
 <td class="td_name sv_use"><img src="http://img.coolenjoy.net/image/o_n.gif" style="vertical-align:middle"/>  생선팔이</td>
 <td class="td_date">20-01-08 </td>
 <td class="td_hit">49</td>
 <td class="td_vote"><div class="list_good"><img src="http://www.coolenjoy.net/image/bbs/v0.gif"/></div><div class="list_good2"><b>0</b></div></td>
 </tr>, <tr class="">
 <td class="td_num">
 <i aria-hidden="true" class="fa fa-question"></i> 질문            </td>
 <td class="td_subject">
 <a href="http://www.coolenjoy.net/bbs/40/7818">
                     밸브인덱스 한국출시하나요??                                    </a>
 </td>
 <td class="td_name sv_use"><img src="http://img.coolenjoy.net/image/n8.gif" style="vertical-align:middle"/>  럽린</td>


## 모든 sibling 내용만 구하기 

In [95]:
for parent in ocul.parents:
    if parent is None:
        print(parent)
    else:
        print(parent.name)

a
td
tr
tbody
table
div
form
div
div
[document]


In [121]:
common_ancestor.select('tr>td>a') # CSS selector path로 원하는 것만 고른다. common_ancestor 이전의 것을 list로 저장하는 과정 필요. 

[<a href="http://www.coolenjoy.net/bbs/40/7819">
                     오디세이G9 존버중인데 제 컴사양 너무 구릴까요?                                    </a>,
 <a href="http://www.coolenjoy.net/bbs/40/7818">
                     밸브인덱스 한국출시하나요??                                    </a>,
 <a href="http://www.coolenjoy.net/bbs/40/7816">
                     오큘 리프트s 블랙아웃 현상 때문에 고통받고 있네요                    <span class="sound_only">댓글</span><span class="cnt_cmt">[1]</span><span class="sound_only">개</span> </a>,
 <a href="http://www.coolenjoy.net/bbs/40/7815">
                     vive hdmi 연결했는데 연결 안됐다 뜹니다.                                    </a>,
 <a href="http://www.coolenjoy.net/bbs/40/7802">
                     파이맥스 8K ,영화감상용으론 4K 프로젝터 150인치 보다 나을까요?                    <span class="sound_only">댓글</span><span class="cnt_cmt">[3]</span><span class="sound_only">개</span> </a>,
 <a href="http://www.coolenjoy.net/bbs/40/7801">
                     오딧세이 플러스를 사려고하는데요                    <span class="sound_only">댓글</